In [9]:
# Load dataset
df = pd.read_csv("aqi_dataset.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'aqi_dataset.csv'

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

# ---------- config ----------
DATA_PATH = "data/aqi_dataset.csv"
MODEL_PATH = "model/aqi_model.pkl"

st.set_page_config(page_title="AQI Prediction System", layout="wide")


# ---------- helpers ----------
def ensure_dirs():
    os.makedirs("data", exist_ok=True)
    os.makedirs("model", exist_ok=True)


def generate_synthetic_dataset(n_samples: int = 2000, seed: int = 42) -> pd.DataFrame:
    np.random.seed(seed)
    df = pd.DataFrame({
        "PM2.5": np.random.uniform(0, 200, n_samples),
        "PM10": np.random.uniform(0, 300, n_samples),
        "NO2": np.random.uniform(0, 150, n_samples),
        "SO2": np.random.uniform(0, 80, n_samples),
        "CO": np.random.uniform(0, 8, n_samples),
        "O3": np.random.uniform(0, 180, n_samples),
        "Temperature": np.random.uniform(-10, 45, n_samples),
        "Humidity": np.random.uniform(20, 95, n_samples),
    })

    # A synthetic but plausible formula for AQI (bounded 0-500)
    df["AQI"] = (
        df["PM2.5"] * 0.9 +
        df["PM10"] * 0.45 +
        df["NO2"] * 0.35 +
        df["SO2"] * 0.25 +
        df["CO"] * 8.0 +
        df["O3"] * 0.5 -
        df["Temperature"] * 0.25 -
        df["Humidity"] * 0.12 +
        np.random.normal(0, 10, n_samples)
    ).clip(0, 500)
    return df


def train_and_save_model(csv_path: str, model_path: str):
    df = pd.read_csv(csv_path)
    X = df.drop(columns=["AQI"])
    y = df["AQI"]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = RandomForestRegressor(n_estimators=200, random_state=42)
    model.fit(X_train_scaled, y_train)

    # Save pipeline (store scaler + model + feature names)
    joblib.dump({"model": model, "scaler": scaler, "features": list(X.columns)}, model_path)

    # Return test sets for immediate evaluation
    y_pred_test = model.predict(X_test_scaled)
    metrics = {
        "mse": float(mean_squared_error(y_test, y_pred_test)),
        "mae": float(mean_absolute_error(y_test, y_pred_test)),
        "rmse": float(mean_squared_error(y_test, y_pred_test, squared=False)),
        "r2": float(r2_score(y_test, y_pred_test)),
    }
    # also return feature importances
    fi = {}
    if hasattr(model, "feature_importances_"):
        for f, v in zip(X.columns, model.feature_importances_):
            fi[f] = float(v)
    return metrics, (X_test, y_test, y_pred_test), fi


def load_model(model_path: str):
    data = joblib.load(model_path)
    return data["model"], data["scaler"], data["features"]


def get_aqi_category(aqi: float):
    a = float(aqi)
    if a <= 50:
        return "Good", "green"
    if a <= 100:
        return "Moderate", "orange"
    if a <= 150:
        return "Unhealthy (Sensitive)", "darkorange"
    if a <= 200:
        return "Unhealthy", "red"
    if a <= 300:
        return "Very Unhealthy", "purple"
    return "Hazardous", "maroon"


# ---------- UI ----------
ensure_dirs()

st.title("🌍 Air Quality Index Prediction System")
st.write("Machine Learning based AQI prediction — dataset, model and visualizations integrated in Python.")

left, right = st.columns([2, 1])

with left:
    st.header("1) Dataset")
    if os.path.exists(DATA_PATH):
        st.success(f"Found dataset: `{DATA_PATH}`")
        df = pd.read_csv(DATA_PATH)
        st.write("Preview (top 5 rows):")
        st.dataframe(df.head())
        st.write(f"Rows: {len(df)}")
    else:
        st.warning(f"Dataset not found at `{DATA_PATH}`.")
        if st.button("Generate synthetic dataset (recommended)"):
            df = generate_synthetic_dataset(n_samples=2000)
            df.to_csv(DATA_PATH, index=False)
            st.success(f"Synthetic dataset saved to `{DATA_PATH}` — {len(df)} rows.")
            st.experimental_rerun()

    st.markdown("---")
    st.header("2) Train / Load Model")
    if os.path.exists(MODEL_PATH):
        st.success(f"Found model: `{MODEL_PATH}`")
        if st.button("Retrain model from dataset"):
            with st.spinner("Retraining model (this may take ~15-60s depending on hardware)..."):
                metrics, test_data, fi = train_and_save_model(DATA_PATH, MODEL_PATH)
            st.success("Model retrained and saved.")
            st.write("Latest training metrics:", metrics)
            st.experimental_rerun()
    else:
        st.info("Model not found. Train model from dataset to enable prediction.")
        if os.path.exists(DATA_PATH):
            if st.button("Train model now"):
                with st.spinner("Training model..."):
                    metrics, test_data, fi = train_and_save_model(DATA_PATH, MODEL_PATH)
                st.success("Model trained and saved to `model/aqi_model.pkl`.")
                st.write("Training metrics:", metrics)
                st.experimental_rerun()
        else:
            st.write("Generate dataset first (top button).")

with right:
    st.header("Quick Actions")
    st.write("Use these to prepare dataset and model for the app.")
    if st.button("Delete dataset (danger)"):
        if os.path.exists(DATA_PATH):
            os.remove(DATA_PATH)
            st.warning("Dataset deleted.")
            st.experimental_rerun()
        else:
            st.info("Dataset not present.")
    if st.button("Delete model (danger)"):
        if os.path.exists(MODEL_PATH):
            os.remove(MODEL_PATH)
            st.warning("Model deleted.")
            st.experimental_rerun()
        else:
            st.info("Model not present.")

st.markdown("---")

# If model and dataset exist -> show prediction UI and metrics
if os.path.exists(DATA_PATH) and os.path.exists(MODEL_PATH):
    # load model, scaler, features
    model_obj = joblib.load(MODEL_PATH)
    model = model_obj["model"]
    scaler = model_obj["scaler"]
    features = model_obj["features"]

    # Prepare X,y for metrics & plots
    df = pd.read_csv(DATA_PATH)
    X = df[features]
    y = df["AQI"].values
    X_scaled = scaler.transform(X)
    y_pred_all = model.predict(X_scaled)

    st.markdown("## Live Prediction")
    col1, col2 = st.columns(2)
    with col1:
        # user inputs for prediction
        inputs = {}
        for f in features:
            default = float(df[f].median()) if f in df.columns else 0.0
            inputs[f] = st.number_input(f"{f}", value=float(default))
    with col2:
        if st.button("Predict AQI for inputs"):
            x_arr = np.array([inputs[f] for f in features]).reshape(1, -1)
            x_scaled = scaler.transform(x_arr)
            pred = model.predict(x_scaled)[0]
            cat, color = get_aqi_category(pred)
            st.metric("Predicted AQI", f"{pred:.2f}")
            st.write(f"Category: **{cat}**")

    st.markdown("## Model Performance (on full dataset)")
    mse = mean_squared_error(y, y_pred_all)
    mae = mean_absolute_error(y, y_pred_all)
    rmse = mean_squared_error(y, y_pred_all, squared=False)
    r2 = r2_score(y, y_pred_all)

    m1, m2, m3, m4 = st.columns(4)
    m1.metric("MSE", f"{mse:.2f}")
    m2.metric("MAE", f"{mae:.2f}")
    m3.metric("RMSE", f"{rmse:.2f}")
    m4.metric("R²", f"{r2:.2f}")

    st.markdown("## Actual vs Predicted (sample)")
    fig, ax = plt.subplots(figsize=(6, 4))
    sample_idx = np.random.choice(len(y), size=min(200, len(y)), replace=False)
    ax.scatter(y[sample_idx], y_pred_all[sample_idx], alpha=0.6)
    ax.plot([0, 500], [0, 500], color="gray", linewidth=1, linestyle="--")
    ax.set_xlabel("Actual AQI")
    ax.set_ylabel("Predicted AQI")
    st.pyplot(fig)

    st.markdown("## Feature importance")
    if hasattr(model, "feature_importances_"):
        fi = pd.Series(model.feature_importances_, index=features).sort_values(ascending=False)
        st.bar_chart(fi)
else:
    st.info("Once you generate the dataset and train the model using the buttons above, prediction and metrics will appear here.")


2026-01-23 02:27:09.533 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-23 02:27:09.536 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-23 02:27:09.536 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-23 02:27:09.537 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-23 02:27:09.541 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-23 02:27:09.542 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-23 02:27:09.543 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-23 02:27:09.544 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar